In [133]:
import numpy as np
from itertools import permutations
from scipy.special import comb, perm

In [150]:
def avg_ari(stocks):
    return np.mean(stocks)

def avg_geo(stocks):
    return np.power(np.prod(stocks), 1.0/len(stocks))

def lst_map(lst):
    return tuple(map(lambda x: '%.2f'%x, lst))

def func(stocks, deltas=[1]):
    m1 = avg_ari(stocks)
    m2 = avg_geo(stocks)
    
    NN = len(stocks)
    nn = len(deltas)
    perm_num = int(perm(NN, nn))
    
    out = np.zeros((perm_num, 6))
    out[:,0], out[:,1] = m1, m2
    iters = 0
    
    for target in permutations(list(range(NN)), nn):
        tmp = stocks.copy()
        for i in range(nn):
            tmp[target[i]] *= deltas[i]
        out[iters, [2,3]] = [avg_ari(tmp), avg_geo(tmp)]
        iters += 1
        
    out[:, 4] = out[:, 2] - out[:, 0]
    out[:, 5] = out[:, 3] - out[:, 1]
    avg_tmp = lst_map(list(np.mean(out,axis=0)))
    std_tmp = lst_map(list(np.std(out,axis =0)))
    print()
    print('算术平均%s，扰动后算数平均%s，delta为%s'%avg_tmp[::2])
    print('几何平均%s，扰动后几何平均%s，delta为%s'%avg_tmp[1::2])
    print('delta之差为%.2f'%(np.mean(out,axis=0)[-2]-np.mean(out,axis=0)[-1]))
    print()
    print('算术方差%s，扰动后算数方差%s，delta方差为%s'%std_tmp[::2])
    print('几何方差%s，扰动后几何方差%s，delta方差为%s'%std_tmp[1::2])
    print()

In [165]:
## rets 为对高瓴股票三年收益乘数分布的估计
rets = np.array([4.0,3,2.7,2.5,2.2,2,1.8,1.5,1.25,0.7])

## DELTAS为波动：比如DELTAS为四个数，则取组合中的任一四个股票给其对应的波动，观察他们对账户的影响
## 最后汇总，计算其均值与方差。
DELTAS = np.array([1.5,1.25,0.8,0.6], dtype=float)
print()
perm_num = int(perm(len(rets), len(DELTAS)))
print('高瓴理论年化收益%.2f, 模拟次数：%s' %(np.power(avg_ari(rets), 1/3)-1, perm_num))
print()

print('扰动的算数平均%s，几何平均为%s' %lst_map([avg_ari(DELTAS), avg_geo(DELTAS)]))
func(rets, DELTAS)


高瓴理论年化收益0.29, 模拟次数：5040

扰动的算数平均1.04，几何平均为0.97

算术平均2.17，扰动后算数平均2.20，delta为0.03
几何平均1.96，扰动后几何平均1.94，delta为-0.02
delta之差为0.05

算术方差0.00，扰动后算数方差0.07，delta方差为0.07
几何方差0.00，扰动后几何方差0.00，delta方差为0.00



# 测试：dX总是比dY大 ！
## delta很小 趋于1
## delta有单个大值 eg 3
## delta有单个小值 eg 0.4
## 牛市
## 熊市
## 正常波动

# Intuition
## geo极度厌恶小值，对大值的反应也不如arith激烈
## geo抗风险能力很弱，有一个股票弄错对结果的影响都很大
## dY的std=0，但这也无法弥补dX与dY之间的差距